In [9]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,NET_TIMEOUT_SECONDS
from od.env.station import BS_PRESET
from od.social.manager import  DynamicSocialGroupBehaviour
import od.misc.statistic as oms

In [29]:
print(os.getcwd())
statistics = []
for i, param in enumerate([False, True]):
    interest_config = InterestConfig(
        DynamicSocialGroupBehaviour.MAX_N_MEMBER,
        20,
        False,
        ResourceAllocatorType.NOMA_OPT,
        param,
        1.4,
        1
    )
    with open("../data/GERNAS/GERNA/{}/object.pickle".format(interest_config.folder()), "rb") as file:
        statistics.append(pickle.load(file))


/home/OD/repos/ProjectSumo/ipynb


In [38]:
[ str(stats.interest_config) for stats in statistics]

['MAX_N_MEMBER(20)-nQoS-nRSU-NOMA_OPT-D1.4-S1',
 'MAX_N_MEMBER(20)-nQoS-yRSU-NOMA_OPT-D1.4-S1']

In [31]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]
    class STATISTIC:
        def Log(data):
            pass
        def Doc(data):
            pass
    class RESULT:
        def Log(data):
            pass
        def Doc(data):
            print(data)
oms.GV = GV
# def RecordPrinter(header):
#     print("time at:{}".format(header.at))
#     print("bs ot:{}".format(header.is_bs_ot))
#     print("src ot:{}".format(header.is_src_ot))
#     print("time bs drop:{}".format(header.time_bs_drop))
#     print("time bs serv:{}".format(header.time_bs_serv))
#     print("time bs tx:{}".format(header.time_bs_tx))
#     print("time bs txq:{}".format(header.time_bs_txq))
#     print("time bs trip:{}".format(header.time_veh_trip))

In [32]:
GV.NET_STATION_CONTROLLER = GV.NET_STATION_CONTROLLER[:2]
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2]

In [33]:
for statistic in statistics:
    for sg in statistic.social_group:
        for record in statistic.sg_header[sg].values():
            for bs in GV.NET_STATION_CONTROLLER:
                if(len(record.time_bs_txq[bs])==0):
                    continue
                if(record.time_bs_txq[bs][-1][1] - record.at >= NET_TIMEOUT_SECONDS):
                    record.is_bs_ot[bs] = True

In [34]:
for i, statistic in enumerate(statistics):
    print("====={}======".format(i))
    for sg in statistic.social_group:
        ot_count = 0
        for header, record in statistic.sg_header[sg].items():
            if(record.is_src_ot > 0 or
               True in [record.is_bs_ot[i] for i in GV.NET_STATION_CONTROLLER] or
               not True in [record.time_bs_serv[i] > 0 for i in GV.NET_STATION_CONTROLLER] or
               not record.time_veh_serv > 0
               ):
                ot_count += 1
        print("{}: {} => {}".format(sg, ot_count, ot_count / max(len(statistic.sg_header[sg]), 1)))


=====0======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 0 => 0.0
DYN_SG_9: 0 => 0.0
=====1======
CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 0 => 0.0
DYN_SG_5: 0 => 0.0
DYN_SG_6: 0 => 0.0
DYN_SG_7: 0 => 0.0
DYN_SG_8: 0 => 0.0
DYN_SG_9: 0 => 0.0


In [19]:
class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len =  195-185 if self.time_serv == -1 else round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [20]:
target_header = "flow_9.3-305"
# target_header = "flow_0.58-99"
for statistic in statistics:
    header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
    print("In {}".format(header_in_sg_list))
    print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
    print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
    print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In ['DYN_SG_6']
C?:False
G?:True
C2G?:False
In ['DYN_SG_6']
C?:False
G?:True
C2G?:False


In [21]:
for i,statistic in enumerate(statistics):
    for sg in statistic.social_group:
        if target_header in statistic.sg_header[sg]:
            header = statistic.sg_header[sg][target_header]
            print("======={}:{}========".format(i,sg))
            print(header)

=======0:DYN_SG_6========
time at:187.979
trips:{'flow_2.62': 0.0049999999999954525}
bs ot:[False, False]
src ot:False
time veh serv:187.982
time veh txq:[[187.979, 187.982]]
time veh tx:[[187.982, 187.983]]
time bs drop:[-1, -1]
time bs serv:[187.996, 187.983]
time bs txq:[[[187.983, 187.995], [187.996, 187.996]], [[187.983, 187.983]]]
time bs tx:[[[187.995, 187.996], [187.996, 187.997]], [[187.983, 187.984]]]

=======1:DYN_SG_6========
time at:187.979
trips:{'flow_2.62': 0.117999999999995}
bs ot:[False, False, False, False, False, False, False]
src ot:False
time veh serv:187.979
time veh txq:[[187.979, 187.979]]
time veh tx:[[187.979, 187.98]]
time bs drop:[-1, -1, -1, -1, -1, -1, -1]
time bs serv:[187.98, 188.096, -1, -1, -1, -1, -1]
time bs txq:[[[187.98, 187.98]], [[187.98, 188.096]], [], [], [], [], []]
time bs tx:[[[187.98, 187.981]], [[188.096, 188.097]], [], [], [], [], []]



In [ ]:
bs_index = 1
social_groups = statistics[0].social_group
print(social_groups)
app_stats_fn = lambda x: statistic.sg_header[social_groups[-1]]
# app_stats_fn = lambda x: statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)
# app_stats_fn = lambda x: x.sg_header[SocialGroup.RCWS]

In [ ]:
bs_serv_list = [[] for _ in statistics]
bs_time_gap = [{} for _ in statistics]
for i, statistic in enumerate(statistics):
    for header, record in app_stats_fn(statistic).items():
        if(len(record.time_bs_txq[bs_index]) > 0):
            bs_serv_list[i].append(Helper(header, record, bs_index))
    bs_serv_list[i].sort(key=lambda x : x.time_gen*1000000+x.time_serv)
    for helper in sorted(bs_serv_list[i],key=lambda x: (x.time_serv * 1000) * 10000000 + x.bits):
        at = str(helper.time_gen)
        to = str(helper.time_serv)
        if(to not in bs_time_gap[i]):
             bs_time_gap[i][to] = {"need": 0, "give": 0}
        if(at not in bs_time_gap[i]):
             bs_time_gap[i][at] = {"need": 0, "give": 0}
        bs_time_gap[i][to]["give"]+=helper.bits
        bs_time_gap[i][at]["need"]+=helper.bits
    
   


In [ ]:
for v in bs_serv_list[0]:
    print(v)

In [ ]:
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    data = []
    for i in range(2):
        if(time in bs_time_gap[i]):
            data.append(bs_time_gap[i][time]["need"])
            data.append(bs_time_gap[i][time]["give"])
        else:
            data.append(0)
            data.append(0)
    if(not sum(data)==0):
        print(time, data)


In [ ]:

remain = [0 for _ in statistics]
for time in [ str(round(i/1000,4))for i in range(185000,195000,1)]:
    for i in range(2):
        if(time in bs_time_gap[i]):
            helper = bs_time_gap[i][time]
            remain[i] += (helper["need"] - helper["give"])
    if(not sum(remain)==0):
        print(time, remain, remain[0]-remain[1])
